In [1]:
import requests, json
from bs4 import BeautifulSoup
import re
from difflib import SequenceMatcher
import urllib3

In [8]:
# Disable InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# Define the brands you're interested in
brands = ['loreal', 'maybelline', 'lakme', 'mamaearth', 'pilgrim', 'mcaffeine', 'plum', 'clinique', 'swiss-beauty', 'blue-heaven', 'mars']

In [3]:
# Define headers for requests
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

Scrape Purplle

In [103]:
def scrape_purplle(brand):
    url = f'https://www.purplle.com/brand/{brand}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_list = []

    # Find all product elements based on the provided HTML structure
    product_elements = soup.find_all('div', class_='pdt2')
    
    for product_element in product_elements:
        product_name_element = product_element.find('div', class_='pro-name')
        product_price_element = product_element.find('span', class_='tx-0')

        if product_name_element and product_price_element:
            product_name = product_name_element.text.strip()
            product_price = product_price_element.text.strip()

            product_list.append({'name': product_name, 'price': product_price})

    return product_list

Scrape Myntra

In [9]:
def scrape_myntra_helper(brand, pageno):
    session = requests.Session()

    url = f'https://www.myntra.com/{brand}?p={pageno}'
    response = session.get(url, headers=headers, allow_redirects=False) 

    soup = BeautifulSoup(response.text, 'html.parser')

    script_element = soup.find_all('script', type='application/ld+json')
    required_data = None

    for script in script_element:
        json_data = json.loads(script.string)
        if json_data['@type'] == 'ItemList':
            required_data = json_data
            break

    product_list = []

    try:
        for item in required_data['itemListElement']: # none type error if the page doesn't have products
            s = requests.Session()
            res = s.get(item['url'], headers=headers, verify=False)

            soup = BeautifulSoup(res.text,"html.parser")

            script = None
            for s in soup.find_all("script"):
                if 'pdpData' in s.text:
                    script = s.get_text(strip=True)
                    break

            data = json.loads(script[script.index('{'):])
            name = data['pdpData']['name']
            price = discounted_price = data["pdpData"]["sizes"][0]["sizeSellerData"][0]["discountedPrice"]
            product_list.append({'name': name, 'price': price})
    except:
        pass
        
    return product_list    

In [10]:
def scrape_myntra(brand):
    product_list = []

    pgno = 1
    while True:
        prods = scrape_myntra_helper(brand, pgno)
        if prods == []:
            break
        
        print(len(prods))

        product_list.extend(prods)
        pgno += 1

    return product_list

Compare Products and their Prices

In [127]:


# Function to clean and preprocess product names
def clean_product_name(name):
    # Remove formatting and symbols
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)
    # Convert to lowercase and remove extra spaces
    name = ' '.join(name.lower().split())
    return name

# Function to calculate the similarity between two product names
def get_name_similarity(name1, name2):
    return SequenceMatcher(None, name1, name2).ratio()

# Function to compare common products and their prices
def compare_products(brand):
    myntra_products = scrape_myntra(brand)
    purplle_products = scrape_purplle(brand)

    print("Products found on myntra ", len(myntra_products))
    print("Products found on purplle ", len(purplle_products))

    better_products = []

    for myntra_product in myntra_products:
        for purplle_product in purplle_products:
            myntra_name = clean_product_name(myntra_product['name'])
            purplle_name = clean_product_name(purplle_product['name'])

            myntra_price = int(myntra_product['price'])
            purplle_price = int(purplle_product['price'])

            similarity_score = get_name_similarity(myntra_name, purplle_name)

            if similarity_score > 0.7 and myntra_price < purplle_price and abs(myntra_price-purplle_price) < 200:  # You can adjust the threshold as needed
                better_products.append({
                    'myntra_name': myntra_product['name'],
                    'purplle_name': purplle_product['name'],
                    'myntra_price': myntra_product['price'],
                    'purplle_price': purplle_product['price'],
                    'similarity_score': similarity_score
                })

    return better_products


In [128]:
# Compare products for each brand
for brand in ['lakme']:
    common_products = compare_products(brand)
    print(f'Better products for {brand}:')
    for product in common_products:
        print(f"Product: {product['myntra_name']} {product['purplle_name']}")
        print(f"Myntra Price: {product['myntra_price']}")
        print(f"Purplle Price: {product['purplle_price']}")
        print()
    print()

Products found on myntra  10
Products found on purplle  56
Better products for lakme:
Product: Lakme 9to5 Primer Matte Lipstick - Deep Wine MP6 Lakme 9TO5 Primer + Matte Lip Color MP6 Deep Wine (3.6 g)
Myntra Price: 302
Purplle Price: 358


